In [1]:
# %load_ext autoreload
# %autoreload 2

import os, sys
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

home_dir = os.environ['HOME']
sys.path.insert(0, os.path.join(home_dir, 'Dropbox/git/CMSC773'))
sys.path.insert(0, os.path.join(home_dir, 'Dropbox/git/bertviz'))
from bertviz import head_view

import dataloader
import tomotopy as tp
from itertools import chain
import tqdm
import pandas as pd
import numpy as np
import slda
import post_classifier
import bow

from matplotlib import pyplot as plt
import seaborn as sns
sns.set_style('dark')

### Load Model

In [2]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
base_save_dir = '/home/hadi/Documents/CL2/deep_stuff/saved_models'
mdoel_save_dir = os.path.join(base_save_dir, 'model')
tokenizer_save_dir = os.path.join(base_save_dir, 'tokenizer')

model_load_dir = os.path.join(mdoel_save_dir, os.listdir(mdoel_save_dir)[0])
tokenizer_load_dir = os.path.join(tokenizer_save_dir, os.listdir(tokenizer_save_dir)[0])

model = BertForSequenceClassification.from_pretrained(model_load_dir, output_attentions=True).to(device)
tokenizer = BertTokenizer.from_pretrained(tokenizer_load_dir)

### Load data

In [4]:
base_dir = 'Documents/CL2/umd_reddit_suicidewatch_dataset_v2/processed_data'
load_dir = os.path.join(home_dir, base_dir)

x_np, y_np = np.load(os.path.join(load_dir, 'test_data.npy'), allow_pickle=True)

x_test = list(map(lambda z: torch.tensor(z, dtype=torch.long, device=device), x_np))
y_test = torch.tensor(y_np, dtype=torch.long, device=device)

## viz

In [5]:
def show_head_view(model, tokenizer, inputs):
   
    with torch.no_grad():
        attention = model(inputs[0], token_type_ids=inputs[1])[-1]
    
    sentence_b_start = inputs[1].tolist()[0].index(1)
    input_id_list = inputs[0].tolist()[0]
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)    
    head_view(attention, tokens, sentence_b_start)

In [6]:
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min',
      jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});

<IPython.core.display.Javascript object>

In [49]:
sample_i = 9015

inputs = list(map(lambda z: z[[sample_i]], x_test))

non_mask_inds = (inputs[1].view(-1) > 0).tolist()
non_masked_inputs = [inputs[0][:, non_mask_inds], inputs[1][:, non_mask_inds]]

sum(non_mask_inds), y_test[sample_i]

(9, tensor([1], device='cuda:0'))

In [50]:
show_head_view(model, tokenizer, non_masked_inputs)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>